### Student Performance Prediction

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [7]:
df = pd.read_csv('../Student Performance Prediction/final_student_data.csv')
df.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,ExamScore,Result
0,1001,17,Male,Asian,Secondary,19.833723,7,1,Moderate,No,No,Yes,No,73.0,Pass
1,1002,18,Female,Asian,Primary,15.408756,0,0,Low,No,No,No,No,76.0,Pass
2,1003,15,Female,Hispanic,Higher Education,4.210570,26,0,Moderate,No,No,No,No,3.0,Fail
3,1004,17,Male,Asian,Higher Education,10.028829,14,0,High,Yes,No,No,No,51.0,Pass
4,1005,17,Male,Asian,Secondary,4.672495,17,1,High,No,No,No,No,32.0,Fail


In [12]:
df.dtypes

StudentID              int64
Age                    int64
Gender                object
Ethnicity             object
ParentalEducation     object
StudyTimeWeekly      float64
Absences               int64
Tutoring               int64
ParentalSupport       object
Extracurricular       object
Sports                object
Music                 object
Volunteering          object
ExamScore            float64
Result                object
dtype: object

In [20]:
df.isna().sum()

StudentID              0
Age                    0
Gender                 0
Ethnicity              0
ParentalEducation    243
StudyTimeWeekly        0
Absences               0
Tutoring               0
ParentalSupport      212
Extracurricular        0
Sports                 0
Music                  0
Volunteering           0
ExamScore              0
Result                 0
dtype: int64

In [21]:
df2 = df.dropna()
df2.isna().sum()

StudentID            0
Age                  0
Gender               0
Ethnicity            0
ParentalEducation    0
StudyTimeWeekly      0
Absences             0
Tutoring             0
ParentalSupport      0
Extracurricular      0
Sports               0
Music                0
Volunteering         0
ExamScore            0
Result               0
dtype: int64

In [39]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ['Gender', 'Ethnicity', 'ParentalEducation', 'ParentalSupport', 'Extracurricular', 'Sports', 'Music', 'Volunteering', 'Result']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot",
                                  one_hot,
                                  categorical_features)],
                                  remainder="passthrough")

transformed_x = transformer.fit_transform(df.drop('ExamScore', axis=1))
transformed_x


array([[ 0.        ,  1.        ,  1.        , ..., 19.83372281,
         7.        ,  1.        ],
       [ 1.        ,  0.        ,  1.        , ..., 15.40875606,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  4.21056977,
        26.        ,  0.        ],
       ...,
       [ 0.        ,  1.        ,  1.        , ...,  6.80549964,
        20.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ..., 12.41665266,
        17.        ,  0.        ],
       [ 0.        ,  1.        ,  1.        , ..., 17.81990749,
        13.        ,  0.        ]])

In [40]:
pd.DataFrame(transformed_x)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1001.0,17.0,19.833723,7.0,1.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1002.0,18.0,15.408756,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1003.0,15.0,4.210570,26.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1004.0,17.0,10.028829,14.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1005.0,17.0,4.672495,17.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,3388.0,18.0,10.680555,2.0,0.0
2388,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,3389.0,17.0,7.583217,4.0,1.0
2389,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,3390.0,16.0,6.805500,20.0,0.0
2390,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,3391.0,16.0,12.416653,17.0,0.0


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# Split into x/y
x = df.drop('ExamScore', axis=1)
x = pd.get_dummies(x)

y = df['ExamScore']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()

In [42]:
model.fit(x_train, y_train)

LinearRegression()

In [43]:
model.score(x_test, y_test)

0.9562789287883451